In [2]:
import polars as pl
pl.Config.with_columns_kwargs = True

In [3]:
pitching = pl.read_csv('./Data/lahman_1871-2023_csv/Pitching.csv')
pitching.head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2


In [67]:
league_era = (
    pitching
    .select(['yearID', 'lgID', 'IPouts', 'R'])
    .filter(pl.col('yearID') >= 1946)
    .group_by([pl.col('yearID'), pl.col('lgID')])
    .agg(pl.col('R').sum().alias('total_runs'),
         pl.col('IPouts').sum().alias('total_outs'))
    .with_columns(era = (pl.col('total_runs') / pl.col('total_outs')) * 27)
    .with_columns(
        pl.when(pl.col('yearID') >= 1973)
          .then(pl.when(pl.col('lgID') == 'AL')
                  .then(pl.lit('yes'))
                  .otherwise(pl.lit('no'))
               )
          .otherwise(pl.lit('no'))
          .alias('DH')
    )
)
league_era.write_csv('data/league_era_percentage.csv')
print(league_era.head(10))


shape: (10, 6)
┌────────┬──────┬────────────┬────────────┬──────────┬─────┐
│ yearID ┆ lgID ┆ total_runs ┆ total_outs ┆ era      ┆ DH  │
│ ---    ┆ ---  ┆ ---        ┆ ---        ┆ ---      ┆ --- │
│ i64    ┆ str  ┆ i64        ┆ i64        ┆ f64      ┆ str │
╞════════╪══════╪════════════╪════════════╪══════════╪═════╡
│ 1971   ┆ NL   ┆ 7601       ┆ 52427      ┆ 3.914529 ┆ no  │
│ 1967   ┆ AL   ┆ 5992       ┆ 43805      ┆ 3.693277 ┆ no  │
│ 1984   ┆ NL   ┆ 7894       ┆ 52274      ┆ 4.077323 ┆ no  │
│ 1966   ┆ AL   ┆ 6276       ┆ 43478      ┆ 3.897419 ┆ no  │
│ 1998   ┆ AL   ┆ 11354      ┆ 60584      ┆ 5.060049 ┆ yes │
│ 1969   ┆ AL   ┆ 7960       ┆ 52510      ┆ 4.092935 ┆ no  │
│ 1964   ┆ AL   ┆ 6607       ┆ 43845      ┆ 4.068628 ┆ no  │
│ 1956   ┆ NL   ┆ 5275       ┆ 33190      ┆ 4.291202 ┆ no  │
│ 1995   ┆ NL   ┆ 9329       ┆ 54168      ┆ 4.650033 ┆ no  │
│ 2023   ┆ AL   ┆ 11110      ┆ 64733      ┆ 4.633958 ┆ yes │
└────────┴──────┴────────────┴────────────┴──────────┴─────┘


In [10]:
batting = pl.read_csv('./Data/lahman_1871-2023_csv/Batting.csv')
batting.head()

playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""aardsda01""",2004,1,"""SFN""","""NL""",11,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2006,1,"""CHN""","""NL""",45,null,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,null
"""aardsda01""",2007,1,"""CHA""","""AL""",25,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2008,1,"""BOS""","""AL""",47,null,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,null
"""aardsda01""",2009,1,"""SEA""","""AL""",73,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null


In [63]:
league_slugging = (
    batting
    .select(['yearID', 'lgID', 'H', '2B', '3B', 'HR', 'AB'])
    .filter(pl.col('yearID') >= 1946)
    .with_columns(
        # Calculate Total Bases
        (pl.col('H') + 
         pl.col('2B') + 
         (2 * pl.col('3B')) + 
         (3 * pl.col('HR'))).alias('Total_Bases')
    )
    .group_by(['yearID', 'lgID'])
    .agg([
        pl.col('Total_Bases').sum().alias('total_bases'),
        pl.col('AB').sum().alias('total_at_bats')
    ])
    .with_columns(
        (pl.col('total_bases') / pl.col('total_at_bats')).alias('Slugging_Percentage')
    )
    .with_columns(
        pl.when(pl.col('yearID') >= 2022)
          .then(pl.lit('yes'))  # DH for both leagues since 2022
          .when(pl.col('yearID') == 2020)
          .then(pl.lit('yes'))  #  
          .when(pl.col('yearID') >= 1973)
          .then(pl.when(pl.col('lgID') == 'AL')
                  .then(pl.lit('yes'))
                  .otherwise(pl.lit('no'))
               )
          .otherwise(pl.lit('no'))
          .alias('DH')
    )
)

league_slugging.write_csv('data/league_slugging_percentage.csv')

print(league_slugging.head())





shape: (5, 6)
┌────────┬──────┬─────────────┬───────────────┬─────────────────────┬─────┐
│ yearID ┆ lgID ┆ total_bases ┆ total_at_bats ┆ Slugging_Percentage ┆ DH  │
│ ---    ┆ ---  ┆ ---         ┆ ---           ┆ ---                 ┆ --- │
│ i64    ┆ str  ┆ i64         ┆ i64           ┆ f64                 ┆ str │
╞════════╪══════╪═════════════╪═══════════════╪═════════════════════╪═════╡
│ 1960   ┆ AL   ┆ 16215       ┆ 41838         ┆ 0.387566            ┆ no  │
│ 1981   ┆ AL   ┆ 18931       ┆ 50813         ┆ 0.372562            ┆ yes │
│ 2001   ┆ AL   ┆ 33450       ┆ 78134         ┆ 0.428111            ┆ yes │
│ 2018   ┆ AL   ┆ 34366       ┆ 82710         ┆ 0.4155              ┆ yes │
│ 1973   ┆ NL   ┆ 24839       ┆ 66087         ┆ 0.375853            ┆ no  │
└────────┴──────┴─────────────┴───────────────┴─────────────────────┴─────┘
